In [10]:
import os

import psycopg2
from dotenv import load_dotenv


load_dotenv()

True

In [15]:
db_params = {
    "host": "127.0.0.1",
    "port": os.environ["POSTGRES_PORT"],
    "database": os.environ["POSTGRES_DB"],
    "user": os.environ["POSTGRES_USER"],
    "password": os.environ["POSTGRES_PASSWORD"],
}

In [16]:
def get_available_tables(schema: str = "public") -> list[str]:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    cursor.execute(
        """
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = %s
        AND table_type = 'BASE TABLE';
        """,
        (schema,),
    )
    tables = [row[0] for row in cursor.fetchall()]
    conn.close()
    if not tables:
        return f"No tables found in schema '{schema}'."
    return "Available tables:\n - " + "\n - ".join(tables)


def get_table_schema(table_name: str = "resumes") -> str:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    cursor.execute(
        """
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = %s;
        """,
        (table_name,),
    )
    columns = cursor.fetchall()
    conn.close()

    schema_lines = [f" - {col[0]} ({col[1]})" for col in columns]
    return f"Table Schema for '{table_name}':\n" + "\n".join(schema_lines)


def get_table_preview(table_name: str = "resumes", limit: int = 5) -> str:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {table_name} LIMIT {limit}")
    rows = cursor.fetchall()
    colnames = [desc[0] for desc in cursor.description]
    conn.close()

    header = " | ".join(colnames)
    lines = [" | ".join(str(cell) for cell in row) for row in rows]

    return f"First {limit} rows from '{table_name}':\n" + header + "\n" + "\n".join(lines)


display(get_available_tables())
display(get_table_schema())
display(get_table_preview())


OperationalError: connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


In [13]:
from smolagents import OpenAIServerModel, Tool, ToolCallingAgent


class SQLExecutorTool(Tool):
    """A tool that executes SQL queries on the 'fake_db.sqlite' database.

    Args:
        query (str): A SQL query to execute.

    Returns:
        string: Query results as a string.

    """

    name = "sql_executor"
    description = "Executes SQL queries on the SQLite database 'fake_db.sqlite'. Returns query results as a string."
    inputs = {"query": {"type": "string", "description": "A valid SQL query string."}}
    output_type = "string"

    def forward(self, query: str):
        output = ""
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()
        cursor.execute(query)

        try:
            output = cursor.fetchall()
        except psycopg2.ProgrammingError:
            output = "Query executed successfully, but no results to fetch."

        conn.commit()
        conn.close()
        return output

In [14]:
model = OpenAIServerModel(
    model_id="qwen2.5:7b",
    api_base="http://localhost:11434/v1",
    api_key="ollama",
    # model_id="large",
    # api_base=os.getenv("AGENT_LLM_API_URL"),
    # api_key=os.getenv("AGENT_LLM_API_TOKEN"),
    # flatten_messages_as_text=True,
)


sql_tool = SQLExecutorTool()
agent = ToolCallingAgent(
    tools=[sql_tool],
    model=model,
    description=f"{get_available_tables(), get_table_schema(), get_table_preview()}\nUse the sql_executor tool to run SQL queries.",
)

prompt = (
    "Сколько резюме devops?. Use the sql_executor tool to run a SQL query."
)

final_output = agent.run(prompt)
print(final_output)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find the name of . Use the sql_executor tool to run a SQL query.                                                │
│                                                                                                                 │
╰─ OpenAIServerModel - qwen2.5:7b ────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error while generating or parsing output:
The model output does not contain any JSON blob.

[Step 1: Duration 45.45 seconds| Input tokens: 1,098 | Output tokens: 52]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

Error while generating or parsing output:
The JSON blob you used is invalid due to the following error: Expecting property name enclosed in double quotes: 
line 1 column 2 (char 1).
JSON blob was: Called Tool: 'sql_executor' with arguments: {'query': 'SELECT name FROM sqlite_master WHERE type = 
"table";'}
, decoding failed on that specific part of the blob:
''.

[Step 3: Duration 12.26 seconds| Input tokens: 3,564 | Output tokens: 177]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 4: Duration 8.30 seconds| Input tokens: 4,886 | Output tokens: 206]

KeyboardInterrupt: 

In [78]:
from openai import AsyncOpenAI
from pydantic_ai import Agent, Tool
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider


def sql_engine(query: str) -> str:
    """Allow to perform SQL queries on the table. Returns a string representation of the result.

    The table is named 'resumes'. Its description is as follows:
        - id (integer)
        - contact_info (jsonb)
        - experience (jsonb)
        - education (jsonb)
        - portfolio (jsonb)
        - languages (ARRAY)
        - skills (ARRAY)
        - certifications (ARRAY)
        - hobbies (ARRAY)
        - name (text)
        - gender (text)
        - occupation (text)
        - summary (text).

    First 5 rows from 'resumes':
        id | name | gender | occupation | summary | contact_info | skills | experience | education | languages | certifications | hobbies | portfolio
        1 | Зыкова Валерия Кузьминична | женский | None | Опытный мобильный разработчик с 5-летним стажем в разработке и оптимизации мобильных приложений. Обладаю глубоким пониманием современных технологий и платформ, таких как iOS и Android. Сильные навыки в проектировании, разработке и тестировании приложений, а также в работе в команде и управлении проектами. Ищу возможность применить свои навыки и знания в динамичной и инновационной компании. | {'email': 'valeriya.zykova@example.com', 'phone': '+7 (808) 262-35-84', 'github': 'github.com/valeriya-zykova', 'linkedin': 'linkedin.com/in/valeriya-zykova', 'location': 'Чехия'} | ['Swift', 'Kotlin', 'React Native', 'Firebase', 'Git', 'Agile/Scrum', 'UX/UI Design', 'RESTful APIs', 'CI/CD', 'Test-Driven Development'] | [{'company': 'TechSolutions', 'end_date': '2023-05-31', 'job_title': 'Mobile Developer', 'start_date': '2018-06-01', 'achievements': ['Разработала и запустила 3 мобильных приложения для iOS и Android, которые достигли более 100 000 загрузок.', 'Оптимизировала производительность приложений, сократив время загрузки на 30%.', 'Внедрила CI/CD пайплайны, что сократило время развертывания на 50%.', 'РаЬотала в мультидисциплинарной команде, координируя усилия разработчиков, дизайнеров и тестировщиков.']}] | [{'degree': 'Бакалавр информационных технологий', 'details': 'Специализация: Программная инженерия', 'end_date': '2017-06-30', 'start_date': '2013-09-01', 'institution': 'Санкт-Петербургский Политехнический Университет'}] | ['Русский – родной', 'Английский – B2', 'Чешский – A2'] | ['Google Certified Professional Cloud Developer', 'Apple Developer Program'] | ['Фотография', 'Путешествия', 'Программирование в свободное время'] | [{'link': 'github.com/valeriya-zykova/TravelApp', 'name': 'TravelApp', 'description': 'Мобильное приложение для планирования путешествий. Использовались технологии: Swift, Firebase, MapKit. Приложение позволяет пользователям создавать маршруты, добавлять точки интереса и делиться планами с друзьями.'}, {'link': 'github.com/valeriya-zykova/FitnessTracker', 'name': 'FitnessTracker', 'description': 'Приложение для отслеживания физической активности. Использовались технологии: Kotlin, Google Fit API, Room Database. Приложение позволяет пользователям отслеживать свои тренировки, устанавливать цели и получать уведомления о прогрессе.'}]
        2 | Фадеева Татьяна Кузьминична | женский | None | Опытный Product Owner с 9-летним стажем в управлении продуктами и командами. Сильные навыки в аджайл-методологиях, стратегическом планировании и взаимодействии с ключевыми заинтересованными сторонами. Успешно запустила и масштабировала несколько продуктов, достигнув значительных бизнес-результатов и улучшив пользовательский опыт. | {'email': 'tatyana.fadeeva@example.com', 'phone': '8 764 180 2753', 'github': 'github.com/tatyana-fadeeva', 'linkedin': 'linkedin.com/in/tatyana-fadeeva', 'location': 'Мали'} | ['Agile и Scrum', 'Product Roadmap', 'User Story Mapping', 'Stakeholder Management', 'Data-Driven Decision Making', 'Leadership', 'Team Building', 'UX/UI Design Basics', 'Jira', 'Confluence'] | [{'company': 'Tech Solutions', 'end_date': '2023', 'job_title': 'Product Owner', 'start_date': '2018', 'achievements': ['Успешно запустила и масштабировала продукт для автоматизации бизнес-процессов, что привело к увеличению эффективности на 30% и росту выручки на 25%.', 'Разработала и внедрила стратегию улучшения пользовательского опыта, что повысило удовлетворенность клиентов на 20%.', 'Создала и поддерживала продукт-бэклог, обеспечивая четкое понимание приоритетов и целей команды.', 'Сотрудничала с ключевыми заинтересованными сторонами, включая руководство и технические команды, для выработки согласованных решений.']}, {'company': 'Innovatech', 'end_date': '2018', 'job_title': 'Product Manager', 'start_date': '2014', 'achievements': ['Управляла жизненным циклом продукта от идеи до запуска, обеспечивая своевременное выполнение всех этапов.', 'Разработала и внедрила методику оценки и анализа пользовательского поведения, что позволило оптимизировать функциональность продукта и повысить конверсию на 15%.', 'Организовала и проводила регулярные встречи с командой разработки, обеспечивая эффективное взаимодействие и выполнение спринтов.']}] | [{'degree': 'Магистр', 'details': 'Менеджмент', 'end_date': '2012', 'start_date': '2007', 'institution': 'Московский Государственный Университет'}] | ['Русский – родной', 'Английский – B2'] | ['Certified Scrum Product Owner (CSPO)', 'Google Analytics – Advanced'] | ['Путешествия', 'Фотография', 'Чтение книг по управлению продуктами'] | [{'link': 'github.com/tatyana-fadeeva/automation-project', 'name': 'Автоматизация бизнес-процессов', 'description': 'Разработка и запуск продукта для автоматизации бизнес-процессов, который повысил эффективность работы компании на 30% и увеличил выручку на 25%.'}, {'link': 'github.com/tatyana-fadeeva/ux-improvement', 'name': 'Улучшение пользовательского опыта', 'description': 'Проект по улучшению пользовательского опыта, который повысил удовлетворенность клиентов на 20% и увеличил конверсию на 15%.'}]
        3 | Горбачева Раиса Александровна | женский | None | Опытный Firmware Engineer с 4 годами опыта в разработке и оптимизации встроенного программного обеспечения для различных устройств. Обладаю глубоким пониманием архитектуры микроконтроллеров и встраиваемых систем, а также сильными навыками в области C/C++ и Python. Имею успешный опыт работы в команде и управления проектами, что позволяет эффективно достигать поставленных целей. | {'email': 'raisa.gorbacheva@example.com', 'phone': '8 940 043 58 95', 'github': 'github.com/raisa-gorbacheva', 'linkedin': 'linkedin.com/in/raisa-gorbacheva', 'location': 'Индонезия'} | ['C/C++', 'Python', 'RTOS (FreeRTOS, Zephyr)', 'Microcontrollers (ARM, AVR)', 'Firmware Development', 'Debugging (JTAG, GDB)', 'Version Control (Git)', 'Agile Methodologies', 'Problem Solving', 'Team Collaboration'] | [{'company': 'Tech Innovations', 'end_date': '2023', 'job_title': 'Firmware Engineer', 'start_date': '2019', 'achievements': ['Разработала и внедрила firmware для IoT-устройств, что позволило увеличить производительность на 20% и снизить энергопотребление на 15%.', 'Успешно завершила проект по миграции legacy firmware на современные микроконтроллеры, сократив время разработки на 30%.', 'Создала и поддерживала документацию по разработке firmware, что улучшило процесс передачи знаний и сократило время наboarding новых сотрудников.']}] | [{'degree': 'Бакалавр', 'details': 'Специальность: Информатика и вычислительная техника', 'end_date': '2018', 'start_date': '2014', 'institution': 'Московский государственный университет'}] | ['Русский – родной', 'Английский – B2'] | ['Certified Firmware Developer (CFD) – 2021'] | ['Программирование микроконтроллеров', 'Путешествия', 'Фотография'] | [{'link': 'github.com/raisa-gorbacheva/iot-gateway-firmware', 'name': 'IoT Gateway Firmware', 'description': 'Разработка firmware для IoT-шлюза на базе ARM Cortex-M4. Использовались FreeRTOS, C/C++, и протоколы MQTT и CoAP. Проект позволил обеспечить надежную связь между устройствами и облачной платформой.'}, {'link': 'github.com/raisa-gorbacheva/low-power-sensor-node', 'name': 'Low-Power Sensor Node', 'description': 'Разработка firmware для низкоэнергетического датчика на базе AVR. Использовались оптимизации энергопотребления, такие как deep sleep режимы и энергоэффективные алгоритмы. Проект позволил увеличить время работы устройства от одной батареи на 50%.'}]
        4 | Маргарита Кирилловна Дорофеева | женский | None | Опытный DevOps Engineer с 2 годами опыта в автоматизации инфраструктуры, оптимизации CI/CD пайплайнов и внедрении DevOps практик. Обладаю сильными навыками в работе с Kubernetes, Docker, и облачными платформами. Способна эффективно сотрудничать с командами разработки и операций, обеспечивая надежность и масштабируемость систем. | {'email': 'margarita.dorofeeva@example.com', 'phone': '8 (234) 422-6002', 'github': 'github.com/margarita-dorofeeva', 'linkedin': 'linkedin.com/in/margarita-dorofeeva', 'location': 'Эстония'} | ['Kubernetes', 'Docker', 'CI/CD (Jenkins, GitLab CI)', 'AWS', 'Terraform', 'Linux', 'Git', 'Python', 'Problem-solving', 'Teamwork'] | [{'company': 'TechSolutions', 'end_date': '2023', 'job_title': 'DevOps Engineer', 'start_date': '2021', 'achievements': ['Автоматизировала CI/CD пайплайны с использованием Jenkins, сократив время развертывания на 30%.', 'Внедрила мониторинг и логирование с помощью Prometheus и ELK Stack, улучшив видимость и управляемость инфраструктуры.', 'Настроила высокодоступную и масштабируемую инфраструктуру на AWS, снизив затраты на 20%.']}] | [{'degree': 'Бакалавр информационных технологий', 'details': 'Специализация: DevOps и облачные технологии', 'end_date': '2021', 'start_date': '2017', 'institution': 'Эстонский Технический Университет'}] | ['Русский – родной', 'Английский – B2'] | ['AWS Certified Solutions Architect – Associate', 'Google Cloud Professional DevOps Engineer'] | ['Программирование на Python', 'Путешествия', 'Фитнес'] | [{'link': 'github.com/margarita-dorofeeva/ci-cd-automation', 'name': 'Автоматизация CI/CD пайплайнов', 'description': 'Проект по автоматизации CI/CD пайплайнов с использованием Jenkins и GitLab CI. Включает скрипты для автоматического тестирования, сборки и развертывания приложений.'}, {'link': 'github.com/margarita-dorofeeva/monitoring-logging', 'name': 'Мониторинг и логирование', 'description': 'Проект по внедрению мониторинга и логирования с использованием Prometheus и ELK Stack. Включает конфигурации для сбора и анализа метрик и логов.'}]
        5 | Большакова Акулина Феликсовна | женский | None | Опытный Site Reliability Engineer с 9-летним стажем в обеспечении высокой доступности и надежности систем. Специализируюсь на автоматизации процессов, мониторинге производительности и оптимизации инфраструктуры. Обладаю сильными навыками в области DevOps и системного администрирования, что позволяет эффективно решать сложные технические задачи и улучшать производительность систем. | {'email': 'akulina.bolshakova@example.com', 'phone': '+7 (710) 484-9177', 'github': 'github.com/akulina-bolshakova', 'linkedin': 'linkedin.com/in/akulina-bolshakova', 'location': 'Коста-Рика'} | ['DevOps', 'CI/CD', 'Kubernetes', 'Docker', 'Prometheus', 'Grafana', 'Terraform', 'AWS', 'Python', 'Linux', 'Problem-solving', 'Agile'] | [{'company': 'TechSolutions Inc.', 'end_date': '2023', 'job_title': 'Site Reliability Engineer', 'start_date': '2018', 'achievements': ['Разработала и внедрила CI/CD пайплайны, что сократило время развертывания на 40%.', 'Автоматизировала мониторинг и оповещения с использованием Prometheus и Grafana, повысив эффективность обнаружения и реагирования на инциденты на 30%.', 'Оптимизировала инфраструктуру с использованием Terraform, снизив затраты на облако на 25%.']}, {'company': 'WebTech Ltd.', 'end_date': '2018', 'job_title': 'System Administrator', 'start_date': '2014', 'achievements': ['Управление и поддержка серверной инфраструктуры на 100+ серверах, обеспечивающих бесперебойную работу критически важных систем.', 'Разработка и внедрение политик безопасности, что снизило количество инцидентов на 50%.', 'Внедрение контейнеризации с использованием Docker, улучшив масштабируемость и производительность приложений.']}] | [{'degree': 'Бакалавр информационных технологий', 'details': 'Специализация: Системное администрирование и сетевые технологии', 'end_date': '2013', 'start_date': '2009', 'institution': 'Национальный университет информационных технологий'}] | ['Русский – родной', 'Английский – C1'] | ['AWS Certified DevOps Engineer – Professional', 'Google Cloud Professional Cloud Architect'] | ['Путешествия', 'Программирование на Python', 'Фотография'] | [{'link': 'github.com/akulina-bolshakova/monitoring-automation', 'name': 'Автоматизация мониторинга и оповещений', 'description': 'Разработка и внедрение системы мониторинга с использованием Prometheus и Grafana для улучшения обнаружения и реагирования на инциденты.'}, {'link': 'github.com/akulina-bolshakova/terraform-optimization', 'name': 'Оптимизация инфраструктуры с использованием Terraform', 'description': 'Проект по оптимизации инфраструктуры с использованием Terraform, снизивший затраты на облако на 25%.'}]

    Args:
        query: The query to perform. This should be correct SQL.

    """
    output = ""
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    cursor.execute(query)

    try:
        output = cursor.fetchall()
    except psycopg2.ProgrammingError:
        output = "Query executed successfully, but no results to fetch."

    conn.commit()
    conn.close()
    return output



openai_provider = OpenAIProvider(
    openai_client=AsyncOpenAI(
        base_url=os.getenv("AGENT_LLM_API_URL"),
        api_key=os.getenv("AGENT_LLM_API_TOKEN"),
    )
)
openai_model = OpenAIModel("large", provider=openai_provider)


sql_tool = Tool(
    function=sql_engine,
    takes_ctx=False,
)


agent = Agent(
    openai_model,
    system_prompt="""
    You are an SQL analyst. For that you are provided with a set of tools.
    You are limited to only providing the information, that you can retrieve with the provided tools.
    """,
    tools=[sql_tool],
)


result = await agent.run("Выведи имена DEVOPS")
print(result.output)

: 

In [59]:
from smolagents import tool


@tool
def sql_engine(query: str) -> str:
    """Allow to perform SQL queries on the table. Returns a string representation of the result.

    The table is named 'resumes'. Its description is as follows:
        - id (integer)
        - contact_info (jsonb)
        - experience (jsonb)
        - education (jsonb)
        - portfolio (jsonb)
        - languages (ARRAY)
        - skills (ARRAY)
        - certifications (ARRAY)
        - hobbies (ARRAY)
        - name (text)
        - gender (text)
        - occupation (text)
        - summary (text).

    First 5 rows from 'resumes':
        id | name | gender | occupation | summary | contact_info | skills | experience | education | languages | certifications | hobbies | portfolio
        1 | Зыкова Валерия Кузьминична | женский | None | Опытный мобильный разработчик с 5-летним стажем в разработке и оптимизации мобильных приложений. Обладаю глубоким пониманием современных технологий и платформ, таких как iOS и Android. Сильные навыки в проектировании, разработке и тестировании приложений, а также в работе в команде и управлении проектами. Ищу возможность применить свои навыки и знания в динамичной и инновационной компании. | {'email': 'valeriya.zykova@example.com', 'phone': '+7 (808) 262-35-84', 'github': 'github.com/valeriya-zykova', 'linkedin': 'linkedin.com/in/valeriya-zykova', 'location': 'Чехия'} | ['Swift', 'Kotlin', 'React Native', 'Firebase', 'Git', 'Agile/Scrum', 'UX/UI Design', 'RESTful APIs', 'CI/CD', 'Test-Driven Development'] | [{'company': 'TechSolutions', 'end_date': '2023-05-31', 'job_title': 'Mobile Developer', 'start_date': '2018-06-01', 'achievements': ['Разработала и запустила 3 мобильных приложения для iOS и Android, которые достигли более 100 000 загрузок.', 'Оптимизировала производительность приложений, сократив время загрузки на 30%.', 'Внедрила CI/CD пайплайны, что сократило время развертывания на 50%.', 'РаЬотала в мультидисциплинарной команде, координируя усилия разработчиков, дизайнеров и тестировщиков.']}] | [{'degree': 'Бакалавр информационных технологий', 'details': 'Специализация: Программная инженерия', 'end_date': '2017-06-30', 'start_date': '2013-09-01', 'institution': 'Санкт-Петербургский Политехнический Университет'}] | ['Русский – родной', 'Английский – B2', 'Чешский – A2'] | ['Google Certified Professional Cloud Developer', 'Apple Developer Program'] | ['Фотография', 'Путешествия', 'Программирование в свободное время'] | [{'link': 'github.com/valeriya-zykova/TravelApp', 'name': 'TravelApp', 'description': 'Мобильное приложение для планирования путешествий. Использовались технологии: Swift, Firebase, MapKit. Приложение позволяет пользователям создавать маршруты, добавлять точки интереса и делиться планами с друзьями.'}, {'link': 'github.com/valeriya-zykova/FitnessTracker', 'name': 'FitnessTracker', 'description': 'Приложение для отслеживания физической активности. Использовались технологии: Kotlin, Google Fit API, Room Database. Приложение позволяет пользователям отслеживать свои тренировки, устанавливать цели и получать уведомления о прогрессе.'}]
        2 | Фадеева Татьяна Кузьминична | женский | None | Опытный Product Owner с 9-летним стажем в управлении продуктами и командами. Сильные навыки в аджайл-методологиях, стратегическом планировании и взаимодействии с ключевыми заинтересованными сторонами. Успешно запустила и масштабировала несколько продуктов, достигнув значительных бизнес-результатов и улучшив пользовательский опыт. | {'email': 'tatyana.fadeeva@example.com', 'phone': '8 764 180 2753', 'github': 'github.com/tatyana-fadeeva', 'linkedin': 'linkedin.com/in/tatyana-fadeeva', 'location': 'Мали'} | ['Agile и Scrum', 'Product Roadmap', 'User Story Mapping', 'Stakeholder Management', 'Data-Driven Decision Making', 'Leadership', 'Team Building', 'UX/UI Design Basics', 'Jira', 'Confluence'] | [{'company': 'Tech Solutions', 'end_date': '2023', 'job_title': 'Product Owner', 'start_date': '2018', 'achievements': ['Успешно запустила и масштабировала продукт для автоматизации бизнес-процессов, что привело к увеличению эффективности на 30% и росту выручки на 25%.', 'Разработала и внедрила стратегию улучшения пользовательского опыта, что повысило удовлетворенность клиентов на 20%.', 'Создала и поддерживала продукт-бэклог, обеспечивая четкое понимание приоритетов и целей команды.', 'Сотрудничала с ключевыми заинтересованными сторонами, включая руководство и технические команды, для выработки согласованных решений.']}, {'company': 'Innovatech', 'end_date': '2018', 'job_title': 'Product Manager', 'start_date': '2014', 'achievements': ['Управляла жизненным циклом продукта от идеи до запуска, обеспечивая своевременное выполнение всех этапов.', 'Разработала и внедрила методику оценки и анализа пользовательского поведения, что позволило оптимизировать функциональность продукта и повысить конверсию на 15%.', 'Организовала и проводила регулярные встречи с командой разработки, обеспечивая эффективное взаимодействие и выполнение спринтов.']}] | [{'degree': 'Магистр', 'details': 'Менеджмент', 'end_date': '2012', 'start_date': '2007', 'institution': 'Московский Государственный Университет'}] | ['Русский – родной', 'Английский – B2'] | ['Certified Scrum Product Owner (CSPO)', 'Google Analytics – Advanced'] | ['Путешествия', 'Фотография', 'Чтение книг по управлению продуктами'] | [{'link': 'github.com/tatyana-fadeeva/automation-project', 'name': 'Автоматизация бизнес-процессов', 'description': 'Разработка и запуск продукта для автоматизации бизнес-процессов, который повысил эффективность работы компании на 30% и увеличил выручку на 25%.'}, {'link': 'github.com/tatyana-fadeeva/ux-improvement', 'name': 'Улучшение пользовательского опыта', 'description': 'Проект по улучшению пользовательского опыта, который повысил удовлетворенность клиентов на 20% и увеличил конверсию на 15%.'}]
        3 | Горбачева Раиса Александровна | женский | None | Опытный Firmware Engineer с 4 годами опыта в разработке и оптимизации встроенного программного обеспечения для различных устройств. Обладаю глубоким пониманием архитектуры микроконтроллеров и встраиваемых систем, а также сильными навыками в области C/C++ и Python. Имею успешный опыт работы в команде и управления проектами, что позволяет эффективно достигать поставленных целей. | {'email': 'raisa.gorbacheva@example.com', 'phone': '8 940 043 58 95', 'github': 'github.com/raisa-gorbacheva', 'linkedin': 'linkedin.com/in/raisa-gorbacheva', 'location': 'Индонезия'} | ['C/C++', 'Python', 'RTOS (FreeRTOS, Zephyr)', 'Microcontrollers (ARM, AVR)', 'Firmware Development', 'Debugging (JTAG, GDB)', 'Version Control (Git)', 'Agile Methodologies', 'Problem Solving', 'Team Collaboration'] | [{'company': 'Tech Innovations', 'end_date': '2023', 'job_title': 'Firmware Engineer', 'start_date': '2019', 'achievements': ['Разработала и внедрила firmware для IoT-устройств, что позволило увеличить производительность на 20% и снизить энергопотребление на 15%.', 'Успешно завершила проект по миграции legacy firmware на современные микроконтроллеры, сократив время разработки на 30%.', 'Создала и поддерживала документацию по разработке firmware, что улучшило процесс передачи знаний и сократило время наboarding новых сотрудников.']}] | [{'degree': 'Бакалавр', 'details': 'Специальность: Информатика и вычислительная техника', 'end_date': '2018', 'start_date': '2014', 'institution': 'Московский государственный университет'}] | ['Русский – родной', 'Английский – B2'] | ['Certified Firmware Developer (CFD) – 2021'] | ['Программирование микроконтроллеров', 'Путешествия', 'Фотография'] | [{'link': 'github.com/raisa-gorbacheva/iot-gateway-firmware', 'name': 'IoT Gateway Firmware', 'description': 'Разработка firmware для IoT-шлюза на базе ARM Cortex-M4. Использовались FreeRTOS, C/C++, и протоколы MQTT и CoAP. Проект позволил обеспечить надежную связь между устройствами и облачной платформой.'}, {'link': 'github.com/raisa-gorbacheva/low-power-sensor-node', 'name': 'Low-Power Sensor Node', 'description': 'Разработка firmware для низкоэнергетического датчика на базе AVR. Использовались оптимизации энергопотребления, такие как deep sleep режимы и энергоэффективные алгоритмы. Проект позволил увеличить время работы устройства от одной батареи на 50%.'}]
        4 | Маргарита Кирилловна Дорофеева | женский | None | Опытный DevOps Engineer с 2 годами опыта в автоматизации инфраструктуры, оптимизации CI/CD пайплайнов и внедрении DevOps практик. Обладаю сильными навыками в работе с Kubernetes, Docker, и облачными платформами. Способна эффективно сотрудничать с командами разработки и операций, обеспечивая надежность и масштабируемость систем. | {'email': 'margarita.dorofeeva@example.com', 'phone': '8 (234) 422-6002', 'github': 'github.com/margarita-dorofeeva', 'linkedin': 'linkedin.com/in/margarita-dorofeeva', 'location': 'Эстония'} | ['Kubernetes', 'Docker', 'CI/CD (Jenkins, GitLab CI)', 'AWS', 'Terraform', 'Linux', 'Git', 'Python', 'Problem-solving', 'Teamwork'] | [{'company': 'TechSolutions', 'end_date': '2023', 'job_title': 'DevOps Engineer', 'start_date': '2021', 'achievements': ['Автоматизировала CI/CD пайплайны с использованием Jenkins, сократив время развертывания на 30%.', 'Внедрила мониторинг и логирование с помощью Prometheus и ELK Stack, улучшив видимость и управляемость инфраструктуры.', 'Настроила высокодоступную и масштабируемую инфраструктуру на AWS, снизив затраты на 20%.']}] | [{'degree': 'Бакалавр информационных технологий', 'details': 'Специализация: DevOps и облачные технологии', 'end_date': '2021', 'start_date': '2017', 'institution': 'Эстонский Технический Университет'}] | ['Русский – родной', 'Английский – B2'] | ['AWS Certified Solutions Architect – Associate', 'Google Cloud Professional DevOps Engineer'] | ['Программирование на Python', 'Путешествия', 'Фитнес'] | [{'link': 'github.com/margarita-dorofeeva/ci-cd-automation', 'name': 'Автоматизация CI/CD пайплайнов', 'description': 'Проект по автоматизации CI/CD пайплайнов с использованием Jenkins и GitLab CI. Включает скрипты для автоматического тестирования, сборки и развертывания приложений.'}, {'link': 'github.com/margarita-dorofeeva/monitoring-logging', 'name': 'Мониторинг и логирование', 'description': 'Проект по внедрению мониторинга и логирования с использованием Prometheus и ELK Stack. Включает конфигурации для сбора и анализа метрик и логов.'}]
        5 | Большакова Акулина Феликсовна | женский | None | Опытный Site Reliability Engineer с 9-летним стажем в обеспечении высокой доступности и надежности систем. Специализируюсь на автоматизации процессов, мониторинге производительности и оптимизации инфраструктуры. Обладаю сильными навыками в области DevOps и системного администрирования, что позволяет эффективно решать сложные технические задачи и улучшать производительность систем. | {'email': 'akulina.bolshakova@example.com', 'phone': '+7 (710) 484-9177', 'github': 'github.com/akulina-bolshakova', 'linkedin': 'linkedin.com/in/akulina-bolshakova', 'location': 'Коста-Рика'} | ['DevOps', 'CI/CD', 'Kubernetes', 'Docker', 'Prometheus', 'Grafana', 'Terraform', 'AWS', 'Python', 'Linux', 'Problem-solving', 'Agile'] | [{'company': 'TechSolutions Inc.', 'end_date': '2023', 'job_title': 'Site Reliability Engineer', 'start_date': '2018', 'achievements': ['Разработала и внедрила CI/CD пайплайны, что сократило время развертывания на 40%.', 'Автоматизировала мониторинг и оповещения с использованием Prometheus и Grafana, повысив эффективность обнаружения и реагирования на инциденты на 30%.', 'Оптимизировала инфраструктуру с использованием Terraform, снизив затраты на облако на 25%.']}, {'company': 'WebTech Ltd.', 'end_date': '2018', 'job_title': 'System Administrator', 'start_date': '2014', 'achievements': ['Управление и поддержка серверной инфраструктуры на 100+ серверах, обеспечивающих бесперебойную работу критически важных систем.', 'Разработка и внедрение политик безопасности, что снизило количество инцидентов на 50%.', 'Внедрение контейнеризации с использованием Docker, улучшив масштабируемость и производительность приложений.']}] | [{'degree': 'Бакалавр информационных технологий', 'details': 'Специализация: Системное администрирование и сетевые технологии', 'end_date': '2013', 'start_date': '2009', 'institution': 'Национальный университет информационных технологий'}] | ['Русский – родной', 'Английский – C1'] | ['AWS Certified DevOps Engineer – Professional', 'Google Cloud Professional Cloud Architect'] | ['Путешествия', 'Программирование на Python', 'Фотография'] | [{'link': 'github.com/akulina-bolshakova/monitoring-automation', 'name': 'Автоматизация мониторинга и оповещений', 'description': 'Разработка и внедрение системы мониторинга с использованием Prometheus и Grafana для улучшения обнаружения и реагирования на инциденты.'}, {'link': 'github.com/akulina-bolshakova/terraform-optimization', 'name': 'Оптимизация инфраструктуры с использованием Terraform', 'description': 'Проект по оптимизации инфраструктуры с использованием Terraform, снизивший затраты на облако на 25%.'}]

    Args:
        query: The query to perform. This should be correct SQL.

    """
    output = ""
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    cursor.execute(query)

    try:
        output = cursor.fetchall()
    except psycopg2.ProgrammingError:
        output = "Query executed successfully, but no results to fetch."

    conn.commit()
    conn.close()
    return output

In [60]:
from smolagents import CodeAgent, OpenAIServerModel


model = OpenAIServerModel(
    model_id=os.getenv("AGENT_LLM_API_MODEL"),
    api_base=os.getenv("AGENT_LLM_API_URL"),
    api_key=os.getenv("AGENT_LLM_API_TOKEN"),
    flatten_messages_as_text=True,
)

agent = CodeAgent(
    tools=[sql_engine],
    model=model,
)
agent.run("Выведи имена DEVOPS")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Выведи имена DEVOPS                                                                                             │
│                                                                                                                 │
╰─ OpenAIServerModel - None ──────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
'NoneType' object has no attribute 'split'

[Step 1: Duration 0.00 seconds]

AgentGenerationError: Error in generating model output:
'NoneType' object has no attribute 'split'